In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import openai
import json
import ijson
import requests
from pywikidata import Entity
import os
openai.api_key = os.getenv('KEY')

openai.base_url = "https://api.vsegpt.ru:6070/v1/"

/Users/aa/vmk/diplom/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
file_path = "lol/RuBQ_2.0_test.json"
with open(file_path, 'r') as file:
    array_items = ijson.items(file, 'item')
    
    i = 0
    true_answers = 0
    answers = []
    for item in array_items:
        messages = []
        i += 1

        question = item["question_eng"]
        try:
            answer = ""
            for x in item["answers"][0]["wd_names"]["en"]:
                answer += x.lower() + ' '
        except:
            continue
        prompt = "Answer shortly in 1-3 words. If answer is a number, print digits"
        # prompt = "Ответь максимально коротко и ёмко на вопрос. Ответ приводи в именительном падеже, при этом не забывай о пунктуации."
        # messages.append({"role": "user", "content": prompt})
        messages.append({"role": "user", "content": prompt + ' ' + question})
        response = ""
        try:
            response_big = openai.chat.completions.create(
                model="openai/gpt-3.5-turbo-1106",
                messages=messages,
                temperature=0.1,
            )
            response = response_big.choices[0].message.content.lower()
        except:
            response = ""
            
        answers.append({
            "question": question,
            "answer": response,
            "correct_answer": answer.lower(),
            "is_correct": response in answer
        })
        if response in answer:
            true_answers += 1
        print(i, true_answers)
        json_data = json.dumps(answers, indent=4)
        with open("questions_answers_rubq.json", "w") as json_file:
            json_file.write(json_data)

In [4]:
def send_request(question_text):
    url = 'http://5.35.10.151:8080/pipeline/act_selection/main'
    headers = {'Content-Type': 'application/json'}
    data = {'text': question_text}

    response = requests.post(url, json=data, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        return None

In [ ]:
response = send_request("Largest country in the world?")
response

In [ ]:
file_path = "lol/RuBQ_2.0_test.json"
with open(file_path, 'r') as file:
    array_items = ijson.items(file, 'item')
    
    i = 144
    true_answers = 51
    answers = []
    for j, item in enumerate(array_items):
        if j < 144:
            continue
        messages = []
        i += 1

        question = item["question_eng"]
        try:
            answer = ""
            for x in item["answers"][0]["wd_names"]["en"]:
                answer += x.lower() + ' '
        except:
            continue
        response_data = send_request(question)
        if not response_data:
            continue

        response = Entity(response_data['answers'][0]).label.lower()
        answers.append({
            "question": question,
            "answer": response,
            "correct_answer": answer.lower(),
            "is_correct": response in answer.lower()
        })
        if response in answer.lower():
            true_answers += 1
        print(i, true_answers)
        json_data = json.dumps(answers[len(answers) - 1], indent=4)
        with open("questions_answers_rubq_act.json", "a") as json_file:
            json_file.write(json_data + ",\n")